In [1]:
import numpy as np
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/full-dataset/Full_dataset/No/train_no (204).png
/kaggle/input/full-dataset/Full_dataset/No/train_no (417).png
/kaggle/input/full-dataset/Full_dataset/No/train_no (654).png
/kaggle/input/full-dataset/Full_dataset/No/test_no (76).png
/kaggle/input/full-dataset/Full_dataset/No/train_no (942).png
/kaggle/input/full-dataset/Full_dataset/No/train_no (944).png
/kaggle/input/full-dataset/Full_dataset/No/train_no (255).png
/kaggle/input/full-dataset/Full_dataset/No/train_no (475).png
/kaggle/input/full-dataset/Full_dataset/No/test_no (89).png
/kaggle/input/full-dataset/Full_dataset/No/train_no (447).png
/kaggle/input/full-dataset/Full_dataset/No/train_no (773).png
/kaggle/input/full-dataset/Full_dataset/No/train_no (741).png
/kaggle/input/full-dataset/Full_dataset/No/train_no (863).png
/kaggle/input/full-dataset/Full_dataset/No/test_no (92).png
/kaggle/input/full-dataset/Full_dataset/No/test_no (178).png
/kaggle/input/full-dataset/Full_dataset/No/train_no (847).png
/kaggle/input/f

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import os
import cv2
import shutil
#import imutils
import itertools

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input

from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools

#init_notebook_mode(connected=True)
RANDOM_SEED = 42


In [3]:
def load_data(dir_path):
    """
    Load resized images as np.arrays to workspace
    """
    X = []
    y = []
    i = 0
    labels = dict()
    for path in tqdm(sorted(os.listdir(dir_path))):
        if not path.startswith('.'):
            labels[i] = path
            for file in os.listdir(dir_path + path):
                if not file.startswith('.'):
                    img = cv2.imread(dir_path + path + '/' + file)
                    X.append(img)
                    y.append(i)
            i += 1
    X = np.array(X)
    y = np.array(y)
    print(f'{len(X)} images loaded from {dir_path} directory.')
    return X, y, labels


In [4]:
train_dir = '../input/full-dataset/Full_dataset/'
X, Y, labels = load_data(train_dir)

100%|██████████| 2/2 [00:15<00:00,  7.80s/it]

2400 images loaded from ../input/full-dataset/Full_dataset/ directory.



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



In [5]:
X[0].shape
X = np.array([cv2.resize(image, (224, 224)) for image in X])

In [6]:
print(labels)
print(X[0].shape)

{0: 'No', 1: 'Yes'}
(224, 224, 3)


In [7]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5, shuffle=True, random_state=1)

In [8]:
cvscores = []
for train, test in kf.split(X, Y):
  
    model = tf.keras.Sequential([
                              tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(224,224,3)),
                              tf.keras.layers.MaxPooling2D(2,2),
    
                              tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
                              tf.keras.layers.MaxPooling2D(2,2),
    
                              tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                              tf.keras.layers.MaxPooling2D(2,2),
    
                              tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                              tf.keras.layers.MaxPooling2D(2,2),
    
                              tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
                              tf.keras.layers.MaxPooling2D(2,2),
    
                              tf.keras.layers.Flatten(),
                              tf.keras.layers.Dense(64, activation='relu'),
                              tf.keras.layers.Dropout(0.40),
                              tf.keras.layers.Dense(128, activation='relu'),
                              tf.keras.layers.Dropout(0.40),
                              tf.keras.layers.Dense(256, activation='relu'),
                              tf.keras.layers.Dropout(0.40),
                              tf.keras.layers.Dense(1, activation='sigmoid')
                              ])
    # Compile model
    model.compile(loss='binary_crossentropy',optimizer=RMSprop(lr=1e-4),metrics=['accuracy'])
    # Fit the model
    model.fit(X[train], Y[train], epochs=50, batch_size=64, verbose=0)
    # evaluate the model
    scores = model.evaluate(X[test], Y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)


2022-03-13 04:24:42.280632: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-13 04:24:42.384033: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-13 04:24:42.384765: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-13 04:24:42.385986: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

accuracy: 96.88%
accuracy: 96.25%
accuracy: 95.83%
accuracy: 95.63%
accuracy: 97.71%


In [9]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

96.46% (+/- 0.76%)
